## Pandas 使用笔记

Python 中处理二维表数据的核心， Python 中的 Excel


In [ ]:
import os
import re

import numpy as np # 当然少不了和 numpy 搭配使用
import pandas as pd

### 基本 io

In [ ]:
# csv 读取中的常见问题
# windows 下的 csv 文件可能是 gb2312 编码，但读取时指定的 encoding 却是 gbk 或者是 gb18030，要多尝试
# 文件路径中如果包含中文，engine 则要指定 python，这是因为 read_csv 方法底层实现是 C 的库
df = pd.read_csv('filepath', encoding='gbk', engine='python')

### 纵向合并某一文件夹内所有的 Excel 文件

经典任务，实现方法有很多，但就是还没有一个统一简洁的官方方法

In [ ]:
def MERGE(workdir):
    # 获取路径内全部文件，这里假设文件夹内有且只有目标的excel文件。
    # 参杂其他东西也是有办法的，但这里先不涉及那么多
    filenames = os.listdir(workdir)
    filepath = []
    for i in range(0,len(filenames)):
        filepath.append(os.path.join(workdir,filenames[i]))
        
    # 遍历获取所有 excel 内容
    content = []
    
    for file in filepath:
        # pandas 读取 excel
        # 根据需要可以增加参数:
        #   sheet_name 指定工作簿 
        #   skipfooter 忽略尾部 
        #   dtype 指定列变量类型，详见文档
        fileread = pd.read_excel(file)

        # 全部加入 content 列表中
        content.append(fileread)
    
    # 将 content 列表中的内容合并为 DataFrame
    result = pd.concat(content) # concat 默认为纵向合并
    
    return result

### 横向合并

同样有多种方法，可以实现类似 sql 的各种 join

In [ ]:
# Merge方法，还有更多的合并参数可以选择，详见文档
df3 = df1.merge(df2, how='outer', on='column_name') # 指定合并根据列
df3 = df1.merge(df2, how='outer', left_index=True, right_index=True) # 根据两侧索引合并

# Join方法，常用于分割列后的返回合并
split = df['column_name'].str.split(expand=True) # 默认按空格分割，且展开列
df = df.join(split)

### 更多具体操作

In [ ]:
df = MERGE('filepath')

# 重设 pandas 默认索引，常用于合并多个 dataframe 之后
df.reset_index(drop=True, inplace=True)

# 使用某一列作为索引
df.set_index([''], inplace=True)

# 输出，可选参数众多，有需要时详见 IO 文档
df.to_excel('filename', sheet_name='', index=True, index_label='', merge_cells=False) # 分割合并单元格
df.to_csv('filename')

# 去除行/列
df.drop(columns=['column_name'], inplace=True) # 关键在于 inplace，此参数默认为 false，只会生成一个新的对象
df.drop(['column_name'], axis=0, inplace=True) # 效果同上，axis=0 即列

# 赋值新建列
df['new_column_name'] = 'new_value'
df['index1'] = list(range(1, len(allmerge['index'])+1)) # 给一个list，新建一个从1开始的索引

# 按某列取值筛选行（类似 excel 的筛选），可组合条件使用
# 注意后面的负号，可以对布尔值取补集
df[(df['column_name1']=='value1' & df['column_name2']=='value2') | df['column_name3'=='value3']]
df = df[df['column_name']!='value' & -df['column_name'].str.contains('str1') & -df['column_name'].str.match('str2')]

# 也可以使用 loc，下面语句筛选出了 number > 0 的 extract 列
df.loc[df['number'] > 0, ['extract']]

# 给定规则替换值，至少有 map 和 replace 两种方法
# map 方法还可以使用函数，更名副其实。详见文档
namemap = {
'被替换值':'替换值',
'中华人民共和国':'中国',
}
df['column_name'] = df['column_name'].map(namemap) # map 方法没有 inplace 参数，只能重新赋值

# replace 方法，这里可以配合正则使用
pattern1 = r'.*身份证$'
pattern2 = r'.*签证$'
# regex 参数用于启动正则模式
df['column_name'].replace( [pattern1, pattern2], ['身份证', '签证'], regex=True, inplace=True)

# 处理文本两侧可能有的多余空格
df['column_name'] = df['column_name'].str.lstrip() # 左去除
df['column_name'] = df['column_name'].str.rstrip() # 右去除

# 处理 csv 中常见的引号文本标记
df['column_name'] = df['column_name'].str.strip('\'')

# 列改名
df.rename(columns = {'old_name':'new_name'}, inplace=True)

# 删除缺失数据
# axis 取 0 即逐行检查各列的缺失情况；how 取 any 即任意一列有缺失都会删除整行
df.dropna(axis=0, how='any')

# 检查空值
pd.isnull(object)

# 缺失值 nan 需要用 numpy 处理
df.replace(to_replace=np.nan, value=0, inplace = True)

# 对列进行函数操作
df['col'].apply(lambda x:2 * x) # 将 col 列中所有值乘上 2

# 数据排序
df.sort_values(ascending=False)

# 索引筛选，假设为时间序列数据。
# 这一部分方法众多，特别是涉及到多层行/列结构的时候，需要时精读文档
df = df.loc['2018-02-08':]

### 日期处理

日期列读入 pandas 后往往只是一般的 object 类型，需要先转换成 datetime 类型方便后续使用

In [ ]:
# Datetime 日期格式转换。
df['new_datetime'] = pd.to_datetime(df['old_datetime'])
# 一次转换多列
df[['time1', 'time2']] = df[['time1', 'time2']].apply(pd.to_datetime)

# Datetime 日期拆解
df['date'] = df['new_datetime'].dt.date
df['time'] = df['new_datetime'].dt.time
df['secs'] = df['datetime1'] - df['datetime2']
df['secs'] = df['secs'].dt.total_seconds()
df[['secs1','secs2']] = df[['secs1','secs2']].astype('timedelta64[h]') # 根据需要调整 timedelta 类型

# Datetime 一般日期列筛选切片（非索引）
# pandas 自身主要的时间序列处理方法都需要将时间列作为索引，限制比较多
# 日期切片，日期的文本字符串可以用'-'或者'/'分割
# df = df[df['datetime'].isin(pd.date_range('2000-01-01','2000-02-01'))]

# 注意：datetime 格式列不能直接使用上面的isin(pd.date_range())方法，只能用下面的比较

# 也可以直接比较字符串日期，但有可能出现类型问题。要用 to_datetime 转换
df = df[df['datetime'] >= '2000-01-01']
df = df[df['datetime'] >= pd.to_datetime('2000-01-01')]
# 注意：传入的字符串默认为 00:00，也就是说日期左闭右开
# 例如，需要 1 月份的日期时
df = df[(df['datetime'] >= pd.to_datetime('2000-01-01')) & (df['datetime'] < pd.to_datetime('2000-02-01'))]  # 正确
df = df[(df['datetime'] >= pd.to_datetime('2000-01-01')) & (df['datetime'] <= pd.to_datetime('2000-01-31'))]  # 错误，会丢掉 1 月 31 日的数据

# 索引列为 datetime 的切片方式
# 日内小时切片
df = df.between_time('23:00','00:00')]

### 数据透视表

类似 Excel 的效果

In [ ]:
# pivot 数据透视表

df_pivot = df.pivot_table(values=['value_column_name'], 
                          index=['row_column_name'], 
                          columns=['column_column_name', 
                                   aggfunc='pd.Series.nunique', 
                                   margins=True)
# 参数：
#   Pandas  | Excel
#   values  |  值
#   index   |  行
#   columns |  列
#   aggfunc | 汇总方法
#  margins-是否显示加总行/列

# pivot 表 stack
df_pivot = df_pivot.stack().to_frame() # 小心 stack()方法生成的是个 series 对象，还需要进一步转化为 dataframe

### 描述性统计

In [ ]:
# 基本的数据集特征信息
df.info()

# 基本的数据集统计信息
df.describe()

# 所有列的情况
df.columns
df.dtypes

# 某列的全部可能取值
df['col'].unique()

# 某列各可能取值的频次统计
df['col'].value_counts()

### 分组处理


In [ ]:
# 分组排名（按数值）：可以处理同分同排名，排名百分比等问题
df.groupby.rank

# 分组排序（按位置）
df.groupby.cumcount

# 分组描述性统计
df.groupby.agg(['mean', 'std'])

# 分组数据应用某项变换
def zscore(x):
    return (x - x.mean()) / x.std()
df.groupby.transform(zscore)